In [34]:
import pandas as pd
import matplotlib.pyplot as plt

In [35]:
md = pd.read_csv("/Users/omarafifi/MyFolders/Differential-Game-Theory-for-SIR-Models/Data/michigan-covid-data.csv")
md['date'] = pd.to_datetime(md['date'])
md = md.set_index('date').sort_index()
md = md[220:]


## Implementing the Adjoint Method

#### 1.1 implement the forward pass 

In [52]:
#step 1 is to solve the forward problem I.e. to integrate the constraining ode. 
timesteps = 21

def SIR(V, t, *args):

    """defines the SIR model
       V = (S, I, R)
       args = (N,Β, γ)
    """

    N, B, g  = args
    S, I, R = V

    dS = -(B/N)*S*I
    dI = (B/N)*S*I - g*I
    dR = g*I

    return dS, dI, dR

def forwardSolve(model, initial_values, params, tf, step):

    """solves the forward problem

       initial_values = (S_0,I_0,R_0)
       params = (N,Β, γ)
       model = SIR
       tf = stop time
       step = number of time steps

    """
    try:
        from scipy.integrate import odeint
        import numpy as np
    except:
        print("unable to install dependancies numpy and scipy.integrate.odeint")
    
    time_steps = np.arange(0, tf, step)
    print(params)

    return odeint(model, t = time_steps, y0 = initial_values, args = params)


    



### 1.2 Backwards integrate  $ \frac{\partial f }{\partial x}  + \lambda (t)^T( \frac{\partial h }{\partial x} - \frac{\partial }{\partial t} \frac{\partial h }{\partial \dot x}) - \frac{\partial \lambda }{\partial t} \frac{\partial h }{\partial \dot x} = 0 $ from $t_f \to 0$